In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Constants
G = 1.0
masses = np.array([1.0, 1.0, 1.0])
t_span = (0, 50)
t_eval = np.linspace(t_span[0], t_span[1], 2000)

# Fixed initial positions for Body 2 and 3
pos_fixed = np.array([[1.0, 0.0], [-1.0, 0.0]])
vel_fixed = np.zeros((2,2))

# Parameters for Body 1 initial grid
x1_vals = np.linspace(-2, 2, 30)
y1_vals = np.linspace(-2, 2, 30)

# Function for 3-body ODE
def three_body_eq(t, y):
    r = y[:6].reshape((3,2))
    v = y[6:].reshape((3,2))
    a = np.zeros((3,2))
    for i in range(3):
        for j in range(3):
            if i != j:
                rij = r[j] - r[i]
                dist3 = np.linalg.norm(rij)**3 + 1e-10
                a[i] += G * masses[j] * rij / dist3
    return np.hstack([v.flatten(), a.flatten()])

# Function to compute entropy of Body 1 for a trajectory
def compute_entropy(pos, vel, nbins=20):
    # Use momentum along x (can generalize later)
    px = vel[0]
    # Compute histogram and normalize to make a probability distribution
    hist, _ = np.histogram(px, bins=nbins, density=False)
    prob = hist / np.sum(hist)  # now sum(prob) = 1
    prob = prob[prob > 0]      # avoid log(0)
    S = -np.sum(prob * np.log(prob))
    return S

# Heat map
entropy_map = np.zeros((len(x1_vals), len(y1_vals)))

for i, x1 in enumerate(x1_vals):
    for j, y1 in enumerate(y1_vals):
        # Initial conditions
        positions = np.vstack([[x1, y1], pos_fixed])
        velocities = np.vstack([np.zeros(2), vel_fixed])
        y0 = np.hstack([positions.flatten(), velocities.flatten()])
        
        # Solve
        sol = solve_ivp(three_body_eq, t_span, y0, t_eval=t_eval, rtol=1e-9, atol=1e-9)
        pos_sol = sol.y[:6].reshape((3,2,-1))
        vel_sol = sol.y[6:].reshape((3,2,-1))
        
        # Compute entropy for Body 1
        S = compute_entropy(pos_sol[0], vel_sol[0])
        entropy_map[i,j] = S

# Plot heat map
plt.figure(figsize=(7,6))
plt.imshow(entropy_map.T, origin='lower', extent=[x1_vals[0], x1_vals[-1], y1_vals[0], y1_vals[-1]],
           aspect='equal', cmap='hot')
plt.colorbar(label='Entropy of Body 1')
plt.xlabel('Initial x1')
plt.ylabel('Initial y1')
plt.title('Phase-Space Entropy vs Initial Position of Body 1')

# Plot fixed positions of Bodies 2 and 3
plt.scatter(pos_fixed[:,0], pos_fixed[:,1], color='cyan', s=80, label='Bodies 2 & 3')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Constants
G = 1.0
masses = np.array([1.0, 1.0, 1.0])
t_span = (0, 50)
t_eval = np.linspace(t_span[0], t_span[1], 2000)

# Grid for initial positions of Body 1
x1_vals = np.linspace(-2, 2, 20)
y1_vals = np.linspace(-2, 2, 20)

# Initial positions and velocities for Bodies 2 and 3 (free to evolve)
pos_init = np.array([[1.0, 0.0], [-1.0, 0.0]])
vel_init = np.array([[0.0, 0.0], [0.0, 0.0]])

# 3-body ODE
def three_body_eq(t, y):
    r = y[:6].reshape((3,2))
    v = y[6:].reshape((3,2))
    a = np.zeros((3,2))
    for i in range(3):
        for j in range(3):
            if i != j:
                rij = r[j] - r[i]
                dist3 = np.linalg.norm(rij)**3 + 1e-10
                a[i] += G * masses[j] * rij / dist3
    return np.hstack([v.flatten(), a.flatten()])

# Function to compute entropy for all bodies
def compute_total_entropy(positions, velocities, nbins=10):
    # Flatten all phase-space coordinates over time
    data = np.vstack([positions.reshape(6,-1), velocities.reshape(6,-1)])  # shape (12, N)
    data = data.T  # shape (N,12)
    
    # Multidimensional histogram
    hist, _ = np.histogramdd(data, bins=nbins)
    prob = hist / np.sum(hist)
    prob = prob[prob > 0]
    
    S = -np.sum(prob * np.log(prob))
    S /= np.log(nbins**12)  # normalize so max entropy = 1
    return S

# Heat map
entropy_map = np.zeros((len(x1_vals), len(y1_vals)))

for i, x1 in enumerate(x1_vals):
    for j, y1 in enumerate(y1_vals):
        # Initial conditions
        positions = np.vstack([[x1, y1], pos_init])
        velocities = np.vstack([[0.0, 0.0], vel_init])
        y0 = np.hstack([positions.flatten(), velocities.flatten()])
        
        # Solve ODE
        sol = solve_ivp(three_body_eq, t_span, y0, t_eval=t_eval, rtol=1e-9, atol=1e-9)
        pos_sol = sol.y[:6].reshape((3,2,-1))
        vel_sol = sol.y[6:].reshape((3,2,-1))
        
        # Compute total entropy
        S = compute_total_entropy(pos_sol, vel_sol, nbins=5)  # smaller bins for speed
        entropy_map[i,j] = S

# Plot heat map
plt.figure(figsize=(7,6))
plt.imshow(entropy_map.T, origin='lower', extent=[x1_vals[0], x1_vals[-1], y1_vals[0], y1_vals[-1]],
           aspect='equal', cmap='hot')
plt.colorbar(label='Normalized phase-space entropy')
plt.xlabel('Initial x1')
plt.ylabel('Initial y1')
plt.title('Global Phase-Space Entropy vs Initial Position of Body 1')

# Optional: plot initial positions of Bodies 2 & 3
plt.scatter(pos_init[:,0], pos_init[:,1], color='cyan', s=80, label='Bodies 2 & 3 init')
plt.legend()
plt.show()